In [74]:
import pandas as pd
import re

In [75]:
# import user rating data
movies_df = pd.read_csv("../raw_data/ml-latest-small/movies.csv")
ratings_df = pd.read_csv("../raw_data/ml-latest-small/ratings.csv")

# import data and create relevant dfs
netflix_df = pd.read_csv("../raw_data/netflix_titles.csv")
# df = pd.read_csv("./titles_sample.csv")
netflix_df['description'] = netflix_df['description'].map(lambda x: re.sub("([^\x00-\x7F])+","", x)) # from jessica's code

In [76]:
# slice years off of titles, remove any quotations
def modify_title(str):
    pattern = r" [(]\d*[)]"
    new_str = re.sub(pattern, '', str)
    new_str = new_str.replace('"', '')
    # move 'The' to beginning of title
    if new_str[-5:] == ", The":
        new_str = "The " + new_str
        new_str = new_str[:-5]
    return new_str

movies_df['title'] = movies_df['title'].apply(modify_title)

In [77]:
# join titles with netflix db movie titles
eval_df = netflix_df.merge(movies_df, on='title', how='left')
# drop titles in netflix that aren't in movielens
eval_df = eval_df.dropna(subset=['movieId'])
# drop irrelvant columns
eval_df.drop(columns=['show_id', 'type', 'director', 'cast', 'country', 'date_added', 
                    'release_year', 'duration', 'rating', 'listed_in', 'genres'], inplace=True)
eval_df = eval_df.reset_index()
eval_df.drop(columns=['index'], inplace=True)
eval_df.drop_duplicates('title', inplace=True)
print(len(eval_df))
eval_df

1058


,title,description,movieId
0,Grown Ups,Mourning the loss of their beloved junior high...,79134.0
1,Dark Skies,A familys idyllic suburban life shatters when ...,100810.0
2,Jaws,When an insatiable great white shark terrorize...,1387.0
3,Jaws 2,Four years after the last deadly shark attacks...,1388.0
4,Jaws: The Revenge,"After another deadly shark attack, Ellen Brody...",4124.0
...,...,...,...
1112,Young Adult,When a divorced writer gets a letter from an o...,91622.0
1113,"Yours, Mine and Ours",When a father of eight and a mother of 10 prep...,26198.0
1115,Zodiac,"A political cartoonist, a crime reporter and a...",51540.0
1116,Zombieland,Looking to survive in a world taken over by zo...,71535.0


In [78]:
# reset movieId and userId so that they are consecutive
codes, uniques = pd.factorize(eval_df['movieId'], sort=True)
num_movies = eval_df['movieId'].nunique()
eval_df['movieId'] = codes
print(num_movies)
print(eval_df['title'].nunique())
eval_df

1058
1058


,title,description,movieId
0,Grown Ups,Mourning the loss of their beloved junior high...,682
1,Dark Skies,A familys idyllic suburban life shatters when ...,793
2,Jaws,When an insatiable great white shark terrorize...,103
3,Jaws 2,Four years after the last deadly shark attacks...,104
4,Jaws: The Revenge,"After another deadly shark attack, Ellen Brody...",283
...,...,...,...
1112,Young Adult,When a divorced writer gets a letter from an o...,750
1113,"Yours, Mine and Ours",When a father of eight and a mother of 10 prep...,447
1115,Zodiac,"A political cartoonist, a crime reporter and a...",530
1116,Zombieland,Looking to survive in a world taken over by zo...,643


In [79]:
eval_df[eval_df['title'] == "The Jungle Book"]

,title,description,movieId
1007,The Jungle Book,This animated series follows young Mowgli and ...,32


In [80]:
# join with ratings_df on movieId to begin evaluation
users_df = eval_df.merge(ratings_df, on='movieId', how='left')
num_users = users_df['userId'].nunique()

codes, uniques = pd.factorize(users_df['userId'], sort=True)
users_df['userId'] = codes
print(num_users)
num_movies = users_df['movieId'].nunique()
print(num_movies)
users_df.drop(columns=['timestamp'], inplace=True)

users_df[users_df['title'] == "The Jungle Book"]

604
1058


,title,description,movieId,userId,rating
20529,The Jungle Book,This animated series follows young Mowgli and ...,32,3,2.0
20530,The Jungle Book,This animated series follows young Mowgli and ...,32,5,4.0
20531,The Jungle Book,This animated series follows young Mowgli and ...,32,7,3.0
20532,The Jungle Book,This animated series follows young Mowgli and ...,32,13,4.0
20533,The Jungle Book,This animated series follows young Mowgli and ...,32,17,4.0
...,...,...,...,...,...
20701,The Jungle Book,This animated series follows young Mowgli and ...,32,596,3.0
20702,The Jungle Book,This animated series follows young Mowgli and ...,32,597,4.0
20703,The Jungle Book,This animated series follows young Mowgli and ...,32,599,4.0
20704,The Jungle Book,This animated series follows young Mowgli and ...,32,601,3.5


In [81]:
# export dfs
users_df.to_csv('../ml_netflix.csv', index=False)